In [3]:
import cv2
import math
import numpy as np
import os
import pm4py
import pandas as pd
from PIL import Image
import numpy as np
import glob

In [24]:
class RandomImgGenerator():
    
    act_dict = None

    def __set_activities(self):
        self.act_dict = {}
        self.activities = sorted(self.activities)
        # проводим соответствие между номером и активити
        act_num = len(self.activities)
        for i in range(act_num):
            self.act_dict[self.activities[i]] = i
                
    
    def __init__(self, activities):
        self.activities = activities
        self.__set_activities()

    def __get_prefix_traces(self):
        cases_prefix_traces = []
        self.max_len_prefix_trace = 0

        for case_log in self.case_logs:
            prefix_traces_act = []
            for i in range(1, len(case_log) + 1):
                prefix_trace = case_log[0:i]
                   
                # находим самую длинную префиксную трассу
                if len(prefix_trace) > self.max_len_prefix_trace:
                    self.max_len_prefix_trace = len(prefix_trace)
                
                
                prefix_traces_act.append(prefix_trace)
                

            cases_prefix_traces.append(prefix_traces_act)
            #print(prefix_traces_act)
        return cases_prefix_traces

    def __get_activity_matrices(self):
        activity_matrices = []
        for prefix_traces in self.cases_prefix_traces:
            np_matrix = []
            matrix = [[0] * len(self.act_dict) for i in range(self.max_len_prefix_trace)]
            for i in range(len(prefix_traces)):
                for act in prefix_traces[i]:
                    act_index = self.act_dict[act]
                    matrix[i][act_index] += 1
            np_matrix = np.asmatrix(matrix)
            np_matrix = np_matrix.astype("uint8")
            activity_matrices.append(np_matrix)
        return activity_matrices
       
    def convert(self, path_to_dir):
        matrix = []
        for i in range(1000):
            row = []
            for j in range(np.random.randint(3, 10)):
                row.append(self.activities[np.random.randint(0, len(self.activities))])
            matrix.append(row)
        
        self.case_logs = matrix
        
        self.cases_prefix_traces = self.__get_prefix_traces()
        self.activity_matrices = self.__get_activity_matrices()

        index = 1
        # freq_matrix = [self.act_freq]  * self.max_len_prefix_trace
        # freq_matrix = np.squeeze(np.asarray(freq_matrix))
        # freq_matrix *= 255.0/(freq_matrix.max()*2)
        # freq_matrix = np.squeeze(np.asarray(freq_matrix))
        # freq_matrix = freq_matrix.astype(int)

        for np_matrix in self.activity_matrices:
           
            norm_matrix = np_matrix.copy()
            norm_matrix *= 255.0 / norm_matrix.max()
            activity_chanel = np.squeeze(np.asarray(norm_matrix))
            activity_chanel = activity_chanel.astype(int)

            result = cv2.merge([activity_chanel, activity_chanel, activity_chanel])
            path = path_to_dir + "/image_" + str(index) + ".png"

            # resize image
            result = cv2.resize(result, (256, 256), interpolation=cv2.INTER_NEAREST)

            cv2.imwrite(path, result)
            # img = Image.fromarray(result)
            # if img.mode != 'RGB':
            #  img = img.convert('RGB')
            # img = img.resize((256, 256), Image.NEAREST)

            index += 1
            # img.save(path)


In [25]:
sorted_act = sorted(['Start', 'Ist', 'A', 'B', 'Icmp', 'T', 'End', 'C', 'E', 'D'])
randomgen = RandomImgGenerator(sorted_act)
randomgen.convert("../data_2/patterns/fraud")